# Self-Attention for transformer

#### 문맥 이해 (Context Understanding)
: Self-Attention은 문장 내의 모든 단어가 서로를 참고할 수 있게 하여, 각 단어의 의미를 문맥에 따라 정확하게 파악할 수 있도록 합니다.



> "The animal didn't cross the street because it was too tired." <br>
여기서 **it** 이 가리키는 대상이 *animal* 인지 *street* 인지 문맥을 고려해서 판단해야 합니다. <br>
Self-Attention은 이런 관계를 파악하는 데 도움을 줍니다.

#### 정보 통합 (Information Integration)
: 각 단어의 표현을 생성할 때, 전체 문장에서 중요한 단어들의 정보를 가중치를 두어 반영함으로써 더 풍부하고 정교한 표현을 만듭니다. <br>
즉, 각 단어를 개별적으로 해석하는 것이 아니라, 문장 전체의 맥락을 고려하여 의미를 구성해 줍니다.


----

### Tokenizer Example

윌리엄 셰익스피어의 The Sonnets(소네트집) 시집을 토큰화하고 Word Bag 으로 만들어 봅니다.

###### 해당 시집은 14행의 소네트 형식을 가진 총 154편의 시로 사랑, 시간, 아름다움, 죽음, 질투, 배신 등 인간의 감정과 삶의 깊은 주제들을 다루고 있습니다.

### Reference: 
1. https://wikidocs.net/166796
2. https://huggingface.co/learn/llm-course/ko/chapter2/4?fw=pt

torchtext 설치시 에러가 발생하여 직접 파싱하는 것으로 대체 <br>
Hunggingface 신경망 라이브러리 설치 필요..

> pip install transformers

In [33]:
import torch
from transformers import AutoTokenizer

# 파일 읽기
with open('shakespeare.txt', 'r') as file:
  text = file.read()

# 토큰나이저 생성
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# 행 단위로 분리 (+소문자로 변환)
lines = text.split('\n')
lines = [line.lower().strip() for line in lines if line.strip()]  # 빈 줄 제거

# 각 행을 토큰화하고 결과를 저장
all_tokens = []
for line in lines:
  all_tokens.extend(line.split())
  # all_tokens.extend(tokenizer.tokenize(line))

    
# 토큰 집합 생성 (중복 제거)
unique_tokens = set(all_tokens)

In [34]:
stoi = {s:i for i, s in enumerate(unique_tokens)}
itos = {i:s for i, s in enumerate(unique_tokens)}

print(stoi)
print(itos)

vocab_size = len(unique_tokens)
print(f"vocabulary size: {vocab_size}")

{'undivided': 0, "learned's": 1, 'spacious,': 2, 'self,': 3, 'become': 4, 'after-loss:': 5, 'thinking': 6, 'over-plus,': 7, 'grecian': 8, 'advance': 9, 'never-resting': 10, 'toward': 11, 'trust,': 12, 'translate!': 13, 'haply': 14, 'comfort,': 15, "truth's": 16, 'these,': 17, 'plagues,': 18, 'policy': 19, 'deserts': 20, 'shalt': 21, 'a': 22, 'wherefore': 23, 'sing:': 24, 'melancholy.': 25, 'praising': 26, 'wherewith': 27, 'rebuked': 28, "love'.": 29, 'commits,': 30, 'offend': 31, 'added': 32, 'lends': 33, 'transfix': 34, 'strange?': 35, 'fester,': 36, 'gored': 37, 'thus.': 38, 'sword,': 39, 'so!': 40, 'stage': 41, 'giving': 42, 'behold,': 43, 'graced': 44, 'bonds': 45, 'clean': 46, 'brought,': 47, 'other,': 48, 'saint': 49, 'thorns,': 50, "intermixed'?": 51, 'taught': 52, 'witness': 53, 'blessing': 54, 'fortune,': 55, 'fury': 56, 'cloak,': 57, 'onward': 58, 'cure,': 59, "men's:": 60, 'heinous': 61, 'limbecks': 62, 'took.': 63, 'invoked': 64, 'uttering': 65, "'truth": 66, 'vows,': 67, '

In [35]:
sentence = "i love you all"
indices = [stoi[word] for word in sentence.split()]
print(indices)

# 단어(토큰)의 정수 인덱스를 학습 가능한 고정된 차원의 연속적인 벡터로 변환
import torch.nn as nn

embedding_dim = 20
# num_embeddings : 단어사전 크기, embedding_dim: 벡터 크기
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

# 모델이 인식할 수 있는 밀집-벡터(dense vector)로 변환
embedded_sentence = embedding(torch.tensor(indices))
print(embedded_sentence)

[2371, 1173, 793, 1026]
tensor([[ 0.2477, -0.3313,  1.1064, -0.6326,  1.3194,  0.3778, -2.0814, -0.1527,
         -1.5948,  1.9727, -1.2865, -0.1500,  0.9934, -0.2072,  1.0374,  0.1523,
         -0.0651,  1.9234, -0.0882,  0.0619],
        [-0.1817, -1.6515,  0.0729, -0.9888, -0.6481, -0.6246, -0.4648,  2.0878,
          1.6984,  1.1344,  1.5739, -0.4028,  0.4333, -1.8122, -0.1278,  1.0623,
          0.3865,  0.0285,  1.5913, -1.3778],
        [-2.1732,  1.2798,  0.5358,  0.9157, -0.1282,  1.6348, -0.4856, -0.3346,
          1.3432,  2.0499,  0.5895, -1.2461, -0.7613, -0.9553, -1.4177, -0.5039,
         -0.2066,  1.3432,  1.1464,  0.3351],
        [ 1.5128,  0.0762,  0.1162, -0.2121, -0.1614,  1.3643, -0.2853,  1.2711,
          0.4427,  0.2444, -0.1347,  0.8433,  0.3167, -0.1238, -0.9987, -0.1086,
         -1.4123, -0.1227, -1.6816,  1.0898]], grad_fn=<EmbeddingBackward0>)
